In [8]:
import numpy as np
import pandas as pd
import torch.nn as nn
import torch
import os
import random

from functions.parse_data import synth_dataloader
from multivariate_quantile_regression.network_model import QuantileNetwork

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [9]:
# Check if CUDA (GPU support) is available
if torch.cuda.is_available():
    # CUDA is available, so let's set default device to GPU
    torch.set_default_device(0)
    print("CUDA is available. Using GPU.")
else:
    # CUDA is not available, so let's use the CPU
    print("CUDA is not available. Using CPU.")

# Example usage:
tensor = torch.randn(3, 3)  # Create a tensor on the selected device
print("Tensor is on device:", tensor.device)

CUDA is available. Using GPU.
Tensor is on device: cuda:0


In [10]:
#Load data and inspect
df = synth_dataloader('SMHIdata2')
df.head(10)

Cloud_B02  Cloud_B03  Cloud_B04  Cloud_B05  Cloud_B06  Cloud_B07  \
0    0.94195    0.87799    0.92936    0.93407    0.95181    0.96217   
1    0.30422    0.40100    0.27834    0.57800    1.01964    1.02787   
2    0.28715    0.25066    0.30366    0.29214    0.34088    0.40079   
3    0.27146    0.33719    0.19841    0.46411    0.88787    0.89584   
4    0.39689    0.38594    0.32623    0.37338    0.60678    0.66895   
5    0.75592    0.65853    0.70670    0.71369    0.73147    0.74182   
6    0.34660    0.30719    0.27459    0.35268    0.72965    0.74705   
7    0.22494    0.33843    0.19615    0.53821    0.98347    0.99256   
8    0.57982    0.62369    0.57909    0.75147    0.99608    1.04055   
9    0.83664    0.80620    0.81432    0.78581    0.81063    0.84108   

   Cloud_B08  Cloud_B08A  Cloud_B09  Cloud_B10  ...  Clear_B11  Clear_B12  \
0    0.92871     0.97181    0.49957    0.04136  ...    0.12946    0.18888   
1    1.00519     1.03599    0.59139    0.01055  ...    0.71532    0.36823   
2    0.37376     0.48750    0.02092    0.00067  ...    0.86232    0.63915   
3    0.87746     0.90439    0.51811    0.00561  ...    0.56307    0.23663   
4    0.55343     0.70168    0.01513    0.00049  ...    0.56472    0.20853   
5    0.71190     0.75003    0.32287    0.00328  ...    0.92187    0.87515   
6    0.69842     0.75893    0.18345    0.00089  ...    0.64499    0.27212   
7    0.96757     1.00132    0.53669    0.00511  ...    0.61388    0.28605   
8    0.94017     1.07212    0.43014    0.03328  ...    0.94942    0.49475   
9    0.78542     0.85153    0.26374    0.00122  ...    0.90206    0.89850   

   Sat_Zenith_Angle  Sun_Zenith_Angle  Azimuth_Diff_Angle     COT  Cloud_Type  \
0              4.53             52.05              167.66   5.897           3   
1             12.85             41.68              161.91   1.275           2   
2             14.53             79.23              168.52   1.799           1   
3              6.54             70.23              165.49   0.519           2   
4              8.56             75.15              148.48   8.569           2   
5             12.38             73.72              153.05  16.874           3   
6             14.74             73.05               13.63   3.589           3   
7              8.24             53.20              146.76   0.624           2   
8             11.57             38.00              117.48  13.909           1   
9              9.69             45.83               23.04  24.937           3   

   Profile_ID    GOT  Water_Vapor  
0        3335  0.126         0.35  
1        1996  0.126         0.31  
2        6796  0.127         4.04  
3        3701  0.123         0.22  
4        6345  0.128         5.40  
5        1419  0.126         0.51  
6         424  0.122         0.99  
7        3427  0.125         0.35  
8        6884  0.122         1.88  
9        7455  0.106         1.77  

[10 rows x 32 columns]

In [11]:
#Set columns for X and y (input/output features)
X_cols = ['Cloud_B02','Cloud_B03','Cloud_B04','Cloud_B05','Cloud_B06',
          'Cloud_B07','Cloud_B08','Cloud_B08A','Cloud_B09','Cloud_B10','Cloud_B11','Cloud_B12',
          'Sat_Zenith_Angle','Sun_Zenith_Angle','Azimuth_Diff_Angle']
y_cols = ['Clear_B02','Clear_B03','Clear_B04','Clear_B05','Clear_B06',
          'Clear_B07','Clear_B08','Clear_B08A','Clear_B09','Clear_B10','Clear_B11','Clear_B12']

#Find X and y
X=df[X_cols]
y=df[y_cols]

#Separate testdata from rest for 80/10/10 Train/Val/Test split
X_trainval, X_test, y_trainval, y_test=train_test_split(X,y,test_size=0.1,random_state=313)

#Add noise to X_test, 0 mean with stdev equal to 3% of mean of each feature
np.random.seed(313)
X_test = X_test + np.random.randn(np.shape(X_test)[0],np.shape(X_test)[1]) * np.mean(X.to_numpy(),axis=0)*0.03

In [12]:
#Set up which quantiles to estimate, and find index of estimator (q=0.5)
quantiles=np.array([0.1,0.5,0.9])
est= np.where(quantiles==0.5)[0].item()

#Set up algorithm parameters for both cases
val_size=0.1
num_models=5 #Set number of models in ensemble
batch_size=500
nepochs=1000
lr=0.003
noise_ratio = 0.03
early_break=True

In [13]:
#Choose if to save models and data, if so set paths
save_load=True
if save_load:
    test_name_1 = "Angles_as_input-with"
    main_filepath_1 = 'pytorch_models/'+test_name_1
    test_name_2 = "Angles_as_input-without"
    main_filepath_2 = 'pytorch_models/'+test_name_2
    test_name_3 = "Angles_as_input-Sun_Zen"
    main_filepath_3 = 'pytorch_models/'+test_name_3

Case 1: Angles included as input features

In [14]:
#Set up NN structure
no_nodes = 100

sequence= lambda: nn.Sequential(
    nn.Linear(len(X_cols),no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes, len(quantiles)*len(y_cols)) #Output dimesion is number of quantiles times number of target variables
)

#Initalize models
models = [QuantileNetwork(quantiles=quantiles) for _ in range(num_models)]

#Train models
for i,model in enumerate(models):
    #Find new train/val splits for each model for robustness
    validation_indices=np.array(random.sample(range(len(X_trainval['Cloud_B02'])), int(len(X['Cloud_B02'])*val_size)))
    train_indices=[i for i in range(len(X_trainval['Cloud_B02'])) if np.any(validation_indices==i)==False]  
    #Fit model
    model.fit(X_trainval.to_numpy(),y_trainval.to_numpy(), 
            train_indices=train_indices, 
            validation_indices=validation_indices, 
            batch_size=batch_size,
            nepochs=nepochs,
            sequence=sequence(),
            lr=lr,
            noise_ratio=noise_ratio,
            early_break=early_break)
    
    #Save models if wanted
    if save_load:
        filepath=main_filepath_1+'/model'+str(i)
        os.makedirs(filepath,exist_ok=True)
        torch.save(model,filepath+'/model_file')
    

Epoch 461


Batch number: 100%|██████████| 320/320 [00:00<00:00, 351.12it/s]

Training loss [2.0786493] Validation loss [2.1393113]
Epoch 462



Batch number: 100%|██████████| 320/320 [00:00<00:00, 353.87it/s]


Training loss [2.0856504] Validation loss [2.1265461]
Epoch 463


Batch number: 100%|██████████| 320/320 [00:00<00:00, 347.48it/s]


Training loss [2.0798266] Validation loss [2.1322474]
---No improvement in 100 epochs, broke early---
Best model out of total max epochs found at epoch 363
With validation loss: 2.110596179962158


In [15]:
#Load models
if save_load:
    base_path = main_filepath_1 + '/'
    model_paths = ['model0/model_file','model1/model_file','model2/model_file','model3/model_file','model4/model_file']
    models = [torch.load(base_path+model_paths[i]) for i in range(len(model_paths))]

#Manually set quantiles
quantiles = np.array([0.1,0.5,0.9])
est = np.where(quantiles==0.5)[0].item()

#Initialize dataframe for error metrics and array for ensemble predictions
with_model_metrics=pd.DataFrame(columns=['Ensemble_mean','Ensemble_index','MSE','PSNR','R2_score','Mean_Quantile_Loss'])
preds_total=[]
#Make predictions and evaluate
for i,model in enumerate(models):
    preds = model.predict(X_test.to_numpy())
    #Keep track of ensemble prediction
    if i==0:
        preds_total=preds
    else:
        preds_total=preds_total+preds

    #Find errors
    mse=mean_squared_error(y_test.to_numpy(),preds[:,:,est])
    psnr=QuantileNetwork.PSNR(y_test.to_numpy(),preds[:,:,est])
    r2=r2_score(y_test.to_numpy(),preds[:,:,est])
    mean_quantile=QuantileNetwork.mean_marginal_loss(y_test.to_numpy(),preds,quantiles)
    #Add to dataframe
    tmp_metrics=pd.DataFrame(data=[[False,i,mse,psnr,r2,mean_quantile]],columns=['Ensemble_mean','Ensemble_index','MSE','PSNR','R2_score','Mean_Quantile_Loss'])
    with_model_metrics=pd.concat([with_model_metrics,tmp_metrics])


#Now do the same for ensemble predictions
preds_total=preds_total/len(models)

mse=mean_squared_error(y_test.to_numpy(),preds_total[:,:,est])
psnr=QuantileNetwork.PSNR(y_test.to_numpy(),preds_total[:,:,est])
r2=r2_score(y_test.to_numpy(),preds_total[:,:,est])
mean_quantile=QuantileNetwork.mean_marginal_loss(y_test.to_numpy(),preds_total,quantiles)

tmp_metrics=pd.DataFrame(data=[[True,np.nan,mse,psnr,r2,mean_quantile]],columns=['Ensemble_mean','Ensemble_index','MSE','PSNR','R2_score','Mean_Quantile_Loss'])
with_model_metrics=pd.concat([with_model_metrics,tmp_metrics])

#Save metrics if we want to
if save_load:
    with_model_metrics=with_model_metrics.reset_index(drop=True)
    with_model_metrics.to_csv(main_filepath_1+'/model_metrics.csv',index=False)
    

/tmp/ipykernel_54489/4082352328.py:30: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  with_model_metrics=pd.concat([with_model_metrics,tmp_metrics])


Case 2: Exclude angles as input features

In [16]:
#Set up NN structure, 3 less in input dim
no_nodes = 100

sequence= lambda: nn.Sequential(
    nn.Linear(len(X_cols)-3,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes, len(quantiles)*len(y_cols)) #Output dimesion is number of quantiles times number of target variables (one)
)

#Initalize models
models = [QuantileNetwork(quantiles=quantiles) for _ in range(num_models)]

#Train models
for i,model in enumerate(models):
    #Find new train/val splits for each model for robustness
    validation_indices=np.array(random.sample(range(len(X_trainval['Cloud_B02'])), int(len(X['Cloud_B02'])*val_size)))
    train_indices=[i for i in range(len(X_trainval['Cloud_B02'])) if np.any(validation_indices==i)==False]  
    #Fit model with y being only band 11
    model.fit(X_trainval.to_numpy()[:,:12],y_trainval.to_numpy(), 
            train_indices=train_indices, 
            validation_indices=validation_indices, 
            batch_size=batch_size,
            nepochs=nepochs,
            sequence=sequence(),
            lr=lr,
            noise_ratio=noise_ratio,
            early_break=early_break)
    
    #Save models if wanted
    if save_load:
        filepath=main_filepath_2+'/model'+str(i)
        os.makedirs(filepath,exist_ok=True)
        torch.save(model,filepath+'/model_file')
    

Epoch 321


Batch number: 100%|██████████| 320/320 [00:00<00:00, 353.90it/s]


Training loss [2.34224] Validation loss [2.3648329]
Epoch 322


Batch number: 100%|██████████| 320/320 [00:00<00:00, 359.26it/s]


Training loss [2.3388112] Validation loss [2.3444405]
Epoch 323


...

Training loss [2.3470492] Validation loss [2.3756452]
Epoch 328


Batch number: 100%|██████████| 320/320 [00:00<00:00, 357.33it/s]


Training loss [2.3529305] Validation loss [2.3650446]
Epoch 329


Batch number: 100%|██████████| 320/320 [00:00<00:00, 353.72it/s]


Training loss [2.344084] Validation loss [2.363338]
---No improvement in 100 epochs, broke early---
Best model out of total max epochs found at epoch 229
With validation loss: 2.3265156745910645


In [ ]:
#Load models
if save_load:
    base_path = main_filepath_2 + '/'
    model_paths = ['model0/model_file','model1/model_file','model2/model_file','model3/model_file','model4/model_file']
    models = [torch.load(base_path+model_paths[i]) for i in range(len(model_paths))]

#Manually set quantiles
quantiles = np.array([0.1,0.5,0.9])
est = np.where(quantiles==0.5)[0].item()

#Initialize dataframe for error metrics and array for ensemble predictions
without_model_metrics=pd.DataFrame(columns=['Ensemble_mean','Ensemble_index','MSE','PSNR','R2_score','Mean_Quantile_Loss'])
preds_total=[]
#Make predictions and evaluate
for i,model in enumerate(models):
    preds = model.predict(X_test.to_numpy()[:,:12])
    #Keep track of ensemble prediction
    if i==0:
        preds_total=preds
    else:
        preds_total=preds_total+preds

    #Find errors
    mse=mean_squared_error(y_test.to_numpy(),preds[:,:,est])
    psnr=QuantileNetwork.PSNR(y_test.to_numpy(),preds[:,:,est])
    r2=r2_score(y_test.to_numpy(),preds[:,:,est])
    mean_quantile=QuantileNetwork.mean_marginal_loss(y_test.to_numpy(),preds,quantiles)
    #Add to dataframe
    tmp_metrics=pd.DataFrame(data=[[False,i,mse,psnr,r2,mean_quantile]],columns=['Ensemble_mean','Ensemble_index','MSE','PSNR','R2_score','Mean_Quantile_Loss'])
    without_model_metrics=pd.concat([without_model_metrics,tmp_metrics])


#Now do the same for ensemble predictions
preds_total=preds_total/len(models)

mse=mean_squared_error(y_test.to_numpy(),preds_total[:,:,est])
psnr=QuantileNetwork.PSNR(y_test.to_numpy(),preds_total[:,:,est])
r2=r2_score(y_test.to_numpy(),preds_total[:,:,est])
mean_quantile=QuantileNetwork.mean_marginal_loss(y_test.to_numpy(),preds_total,quantiles)

tmp_metrics=pd.DataFrame(data=[[True,np.nan,mse,psnr,r2,mean_quantile]],columns=['Ensemble_mean','Ensemble_index','MSE','PSNR','R2_score','Mean_Quantile_Loss'])
without_model_metrics=pd.concat([without_model_metrics,tmp_metrics])

#Save metrics if we want to
if save_load:
    without_model_metrics=without_model_metrics.reset_index(drop=True)
    without_model_metrics.to_csv(main_filepath_2+'/model_metrics.csv',index=False)

/tmp/ipykernel_147790/3042881213.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  without_model_metrics=pd.concat([without_model_metrics,tmp_metrics])


Case 3: Only include Sun Zenith Angle

In [ ]:
#Set up NN structure, 2 less in input dim
no_nodes = 100

sequence= lambda: nn.Sequential(
    nn.Linear(len(X_cols)-2,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes, len(quantiles)*len(y_cols)) #Output dimesion is number of quantiles times number of target variables (one)
)

#Initalize models
models = [QuantileNetwork(quantiles=quantiles) for _ in range(num_models)]

#Train models
for i,model in enumerate(models):
    #Find new train/val splits for each model for robustness
    validation_indices=np.array(random.sample(range(len(X_trainval['Cloud_B02'])), int(len(X['Cloud_B02'])*val_size)))
    train_indices=[i for i in range(len(X_trainval['Cloud_B02'])) if np.any(validation_indices==i)==False]  
    #Fit model with y being only band 11
    model.fit(X_trainval.to_numpy()[:,[0,1,2,3,4,5,6,7,8,9,10,11,13]],y_trainval.to_numpy(), 
            train_indices=train_indices, 
            validation_indices=validation_indices, 
            batch_size=batch_size,
            nepochs=nepochs,
            sequence=sequence(),
            lr=lr,
            noise_ratio=noise_ratio,
            early_break=early_break)
    
    #Save models if wanted
    if save_load:
        filepath=main_filepath_3+'/model'+str(i)
        os.makedirs(filepath,exist_ok=True)
        torch.save(model,filepath+'/model_file')

Epoch 421


Batch number: 100%|██████████| 320/320 [00:00<00:00, 341.24it/s]

Training loss [2.1007407] Validation loss [2.0930197]
Epoch 422



Batch number: 100%|██████████| 320/320 [00:01<00:00, 292.86it/s]


Training loss [2.106531] Validation loss [2.1252537]
Epoch 423


Batch number: 100%|██████████| 320/320 [00:01<00:00, 292.22it/s]


Training loss [2.1000469] Validation loss [2.0998294]
Epoch 424


Batch number: 100%|██████████| 320/320 [00:01<00:00, 292.74it/s]


Training loss [2.0997488] Validation loss [2.0832274]
---No improvement in 100 epochs, broke early---
Best model out of total max epochs found at epoch 324
With validation loss: 2.071849822998047


In [ ]:
#Load models
if save_load:
    base_path = main_filepath_3 + '/'
    model_paths = ['model0/model_file','model1/model_file','model2/model_file','model3/model_file','model4/model_file']
    models = [torch.load(base_path+model_paths[i]) for i in range(len(model_paths))]

#Manually set quantiles
quantiles = np.array([0.1,0.5,0.9])
est = np.where(quantiles==0.5)[0].item()

#Initialize dataframe for error metrics and array for ensemble predictions
sunzen_model_metrics=pd.DataFrame(columns=['Ensemble_mean','Ensemble_index','MSE','PSNR','R2_score','Mean_Quantile_Loss'])
preds_total=[]
#Make predictions and evaluate
for i,model in enumerate(models):
    preds = model.predict(X_test.to_numpy()[:,[0,1,2,3,4,5,6,7,8,9,10,11,13]])
    #Keep track of ensemble prediction
    if i==0:
        preds_total=preds
    else:
        preds_total=preds_total+preds

    #Find errors
    mse=mean_squared_error(y_test.to_numpy(),preds[:,:,est])
    psnr=QuantileNetwork.PSNR(y_test.to_numpy(),preds[:,:,est])
    r2=r2_score(y_test.to_numpy(),preds[:,:,est])
    mean_quantile=QuantileNetwork.mean_marginal_loss(y_test.to_numpy(),preds,quantiles)
    #Add to dataframe
    tmp_metrics=pd.DataFrame(data=[[False,i,mse,psnr,r2,mean_quantile]],columns=['Ensemble_mean','Ensemble_index','MSE','PSNR','R2_score','Mean_Quantile_Loss'])
    sunzen_model_metrics=pd.concat([sunzen_model_metrics,tmp_metrics])


#Now do the same for ensemble predictions
preds_total=preds_total/len(models)

mse=mean_squared_error(y_test.to_numpy(),preds_total[:,:,est])
psnr=QuantileNetwork.PSNR(y_test.to_numpy(),preds_total[:,:,est])
r2=r2_score(y_test.to_numpy(),preds_total[:,:,est])
mean_quantile=QuantileNetwork.mean_marginal_loss(y_test.to_numpy(),preds_total,quantiles)

tmp_metrics=pd.DataFrame(data=[[True,np.nan,mse,psnr,r2,mean_quantile]],columns=['Ensemble_mean','Ensemble_index','MSE','PSNR','R2_score','Mean_Quantile_Loss'])
sunzen_model_metrics=pd.concat([sunzen_model_metrics,tmp_metrics])

#Save metrics if we want to
if save_load:
    sunzen_model_metrics=sunzen_model_metrics.reset_index(drop=True)
    sunzen_model_metrics.to_csv(main_filepath_3+'/model_metrics.csv',index=False)

/tmp/ipykernel_197993/388508426.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  sunzen_model_metrics=pd.concat([sunzen_model_metrics,tmp_metrics])


Display results:

In [ ]:
#Display results with angles
if save_load:
    file_name = main_filepath_1 + '/model_metrics.csv'
    with_model_metrics=pd.read_csv(file_name)
    
with_model_metrics

Ensemble_mean Ensemble_index       MSE       PSNR  R2_score  \
0         False              0  0.006911  21.633462  0.843384   
1         False              1  0.006869  21.659928  0.843847   
2         False              2  0.006953  21.606712  0.842696   
3         False              3  0.006856  21.667865  0.844638   
4         False              4  0.007006  21.573897  0.841085   
5          True            NaN  0.006490  21.906343  0.852975   

   Mean_Quantile_Loss  
0            0.438998  
1            0.435985  
2            0.436534  
3            0.438576  
4            0.438437  
5            0.418722

In [ ]:
#Display results without angles
if save_load:
    file_name = main_filepath_2 + '/model_metrics.csv'
    without_model_metrics=pd.read_csv(file_name)
    
without_model_metrics

Ensemble_mean Ensemble_index       MSE       PSNR  R2_score  \
0         False              0  0.008242  20.868457  0.809809   
1         False              1  0.008185  20.898331  0.810577   
2         False              2  0.008206  20.887634  0.809990   
3         False              3  0.008186  20.898052  0.810647   
4         False              4  0.008228  20.875906  0.809692   
5          True            NaN  0.007735  21.144368  0.821311   
0          True            NaN  0.006499  21.899997  0.852879   

   Mean_Quantile_Loss  
0            0.487105  
1            0.486688  
2            0.485302  
3            0.483422  
4            0.483046  
5            0.465506  
0            0.421465

In [ ]:
#Display results with sun zenith angle
if save_load:
    file_name = main_filepath_3 + '/model_metrics.csv'
    sunzen_model_metrics=pd.read_csv(file_name)
sunzen_model_metrics

Ensemble_mean Ensemble_index       MSE       PSNR  R2_score  \
0         False              0  0.006812  21.695823  0.845446   
1         False              1  0.006972  21.595004  0.841873   
2         False              2  0.006858  21.666677  0.844476   
3         False              3  0.006793  21.708013  0.846127   
4         False              4  0.006931  21.620881  0.843294   
5          True            NaN  0.006471  21.919294  0.853453   

   Mean_Quantile_Loss  
0            0.438479  
1            0.437508  
2            0.439852  
3            0.438455  
4            0.439390  
5            0.419976